In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('/Users/jashanjeetsingh/Downloads/train-balanced-sarcasm.csv')
df.head() Tokenize the text
df['tokenized_text'] = df['text'].apply(lambda x: x.split())

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [6]:
data = {
    'comment': [
        "You're right we should invade now, we'll be done before winter comes.",
        'Because no stock Android',
        'Wow, what a great idea!',
        'Sure, because that worked out so well last time.',
        'I love waiting in long lines, said no one ever.'
    ],
    'label': [1, 0, 1, 1, 1]  # 1 for sarcastic, 0 for non-sarcastic
}

In [8]:
# Ensure all comments are strings
df['comment'] = df['comment'].astype(str)

In [9]:
# Tokenize the text
df['tokenized_text'] = df['comment'].apply(lambda x: x.split())


In [10]:
# Train Word2Vec model
w2v_model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)


In [11]:
# Function to get the vector representation of a text
def get_text_vector(text):
    vectors = [w2v_model.wv[word] for word in text if word in w2v_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)


In [12]:
# Apply the function to all texts
df['vector'] = df['tokenized_text'].apply(get_text_vector)


In [13]:
# Prepare the data for training
X = np.vstack(df['vector'].values)
y = df['label'].values

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Training the Gradient Boosting Machine
gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)


GradientBoostingClassifier()

In [16]:
# Making predictions
y_pred = gbm.predict(X_test)

In [17]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6481356904721862


In [ ]:
#Therefore, we get an accuracy of 64.8%